In [2]:
import argparse
import os
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras

In [3]:
model = ""
mfcc = False
silence = False
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [4]:
# if silence is True:
#     data_dir = os.path.join('.', 'data', 'mini_speech_commands_silence')
# else:
#     zip_path = tf.keras.utils.get_file(
#         origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
#         fname='mini_speech_commands.zip',
#         extract=True,
#         cache_dir='.', cache_subdir='data')
# 
    #  data_dir = os.path('E:\Github\Machine-learning-for-IOT\Lab3\ex2\data\mini_speech_commands')
data_dir = 'E:\Github\Machine-learning-for-IOT\Lab3\ex2\data\mini_speech_commands'

In [5]:
filenames = tf.io.gfile.glob(data_dir + '/*/*')
filenames = tf.random.shuffle(filenames)
num_samples = len(filenames)
if silence is True:
    total = 9000
else:
    total = 8000

In [6]:
num_samples

8000

In [7]:
train_files = filenames[:int(total*0.8)]
val_files = filenames[int(total*0.8): int(total*0.9)]
test_files = filenames[int(total*0.9):]

In [8]:
train_files

<tf.Tensor: shape=(6400,), dtype=string, numpy=
array([b'E:\\Github\\Machine-learning-for-IOT\\Lab3\\ex2\\data\\mini_speech_commands\\up\\b7a6f709_nohash_0.wav',
       b'E:\\Github\\Machine-learning-for-IOT\\Lab3\\ex2\\data\\mini_speech_commands\\go\\e7ea8b76_nohash_7.wav',
       b'E:\\Github\\Machine-learning-for-IOT\\Lab3\\ex2\\data\\mini_speech_commands\\up\\f33660af_nohash_1.wav',
       ...,
       b'E:\\Github\\Machine-learning-for-IOT\\Lab3\\ex2\\data\\mini_speech_commands\\down\\6cb6eee7_nohash_0.wav',
       b'E:\\Github\\Machine-learning-for-IOT\\Lab3\\ex2\\data\\mini_speech_commands\\left\\68effe85_nohash_0.wav',
       b'E:\\Github\\Machine-learning-for-IOT\\Lab3\\ex2\\data\\mini_speech_commands\\stop\\3a7d8872_nohash_0.wav'],
      dtype=object)>

In [9]:
LABELS = np.array(tf.io.gfile.listdir(data_dir))
LABELS = LABELS[LABELS != 'README.md']

In [10]:
LABELS

array(['down', 'go', 'left', 'no', 'right', 'stop', 'up', 'yes'],
      dtype='<U9')

In [51]:
class SignalGenerator:
    def __init__(self, labels, sampling_rate, frame_length, frame_step,
            num_mel_bins=None, lower_frequency=None, upper_frequency=None,
            num_coefficients=None, mfcc=False):
        self.labels = labels
        self.sampling_rate = sampling_rate
        self.frame_length = frame_length
        self.frame_step = frame_step
        self.num_mel_bins = num_mel_bins
        self.lower_frequency = lower_frequency
        self.upper_frequency = upper_frequency
        self.num_coefficients = num_coefficients
        num_spectrogram_bins = (frame_length) // 2 + 1

        if mfcc is True:
            self.linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
                    self.num_mel_bins, num_spectrogram_bins, self.sampling_rate,
                    self.lower_frequency, self.upper_frequency)
            self.preprocess = self.preprocess_with_mfcc
        else:
            self.preprocess = self.preprocess_with_stft

    def read(self, file_path):
        parts = tf.strings.split(file_path, os.path.sep)
        label = parts[-2]
        label_id = tf.argmax(label == self.labels)
        label_id = float(label_id)
        audio_binary = tf.io.read_file(file_path)
        audio, _ = tf.audio.decode_wav(audio_binary)
        audio = tf.squeeze(audio, axis=1)

        return audio, label_id

    def pad(self, audio):
        zero_padding = tf.zeros([self.sampling_rate] - tf.shape(audio), dtype=tf.float32)
        audio = tf.concat([audio, zero_padding], 0)
        audio.set_shape([self.sampling_rate])

        return audio

    def get_spectrogram(self, audio):
        stft = tf.signal.stft(audio, frame_length=self.frame_length,
                frame_step=self.frame_step, fft_length=self.frame_length)
        spectrogram = tf.abs(stft)

        return spectrogram

    def get_mfccs(self, spectrogram):
        mel_spectrogram = tf.tensordot(spectrogram,
                self.linear_to_mel_weight_matrix, 1)
        log_mel_spectrogram = tf.math.log(mel_spectrogram + 1.e-6)
        mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)
        mfccs = mfccs[..., :self.num_coefficients]

        return mfccs

    def preprocess_with_stft(self, file_path):
        audio, label = self.read(file_path)
        audio = self.pad(audio)
        spectrogram = self.get_spectrogram(audio)
        spectrogram = tf.expand_dims(spectrogram, -1)
        spectrogram = tf.image.resize(spectrogram, [32, 32])

        return spectrogram, label

    def preprocess_with_mfcc(self, file_path):
        audio, label = self.read(file_path)
        audio = self.pad(audio)
        spectrogram = self.get_spectrogram(audio)
        mfccs = self.get_mfccs(spectrogram)
        mfccs = tf.expand_dims(mfccs, -1)

        return mfccs, label

    def make_dataset(self, files, train):
        ds = tf.data.Dataset.from_tensor_slices(files)
        ds = ds.map(self.preprocess, num_parallel_calls=4)
        ds = ds.batch(32)
        ds = ds.cache()
        if train is True:
            ds = ds.shuffle(100, reshuffle_each_iteration=True)

        return ds


In [52]:
STFT_OPTIONS = {'frame_length': 256, 'frame_step': 128, 'mfcc': False}
MFCC_OPTIONS = {'frame_length': 640, 'frame_step': 320, 'mfcc': True,
        'lower_frequency': 20, 'upper_frequency': 4000, 'num_mel_bins': 40,
        'num_coefficients': 10}

In [53]:
if mfcc is True:
    options = MFCC_OPTIONS
    strides = [2, 1]
else:
    options = STFT_OPTIONS
    strides = [2, 2]

In [54]:
generator = SignalGenerator(LABELS, 16000, **options)
train_ds = generator.make_dataset(train_files, True)
val_ds = generator.make_dataset(val_files, False)
test_ds = generator.make_dataset(test_files, False)

In [55]:
myit = iter(train_ds)
next(myit)

(<tf.Tensor: shape=(32, 32, 32, 1), dtype=float32, numpy=
 array([[[[2.59880126e-02],
          [1.39740855e-01],
          [1.97291113e-02],
          ...,
          [6.38160557e-02],
          [5.20315096e-02],
          [4.68143746e-02]],
 
         [[1.85705069e-02],
          [1.47200823e-01],
          [2.97472309e-02],
          ...,
          [1.18077733e-02],
          [3.82213201e-03],
          [2.70572468e-03]],
 
         [[3.95819806e-02],
          [3.83832842e-01],
          [1.62676796e-02],
          ...,
          [1.12298399e-03],
          [2.14433437e-03],
          [2.50947056e-03]],
 
         ...,
 
         [[2.00143337e+00],
          [4.34400380e-01],
          [2.75240034e-01],
          ...,
          [6.12634758e-04],
          [1.88128301e-03],
          [6.41707273e-04]],
 
         [[1.47510469e+00],
          [9.38168764e-01],
          [1.54186040e-01],
          ...,
          [1.80140533e-03],
          [6.06244721e-04],
          [8.41947622e-04]]

In [56]:
ll = 0
for i in train_ds:
    print(i)
    if(ll == 0):
        break

(<tf.Tensor: shape=(32, 32, 32, 1), dtype=float32, numpy=
array([[[[6.43769562e-01],
         [2.45258957e-02],
         [7.20597804e-02],
         ...,
         [1.82332867e-03],
         [6.16486184e-04],
         [1.50359556e-04]],

        [[6.15858018e-01],
         [3.29511687e-02],
         [7.66204894e-02],
         ...,
         [5.35138883e-03],
         [1.75584108e-03],
         [3.70878261e-04]],

        [[6.40095711e-01],
         [5.30064888e-02],
         [1.03291556e-01],
         ...,
         [4.50621592e-03],
         [1.25793193e-03],
         [3.23144224e-04]],

        ...,

        [[5.76239347e-01],
         [4.37720083e-02],
         [4.82247211e-02],
         ...,
         [2.08153622e-03],
         [5.37170679e-04],
         [2.72878504e-04]],

        [[5.52905500e-01],
         [7.22492710e-02],
         [6.29498288e-02],
         ...,
         [2.11823545e-03],
         [4.24002181e-04],
         [2.02515876e-04]],

        [[5.79266667e-01],
         [4

In [16]:
# if args.silence is True:
#     units = 9
# else:
#     units = 8

In [17]:
MLPmodel = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(8)
])

In [18]:
#MLPmodel.summary()

In [19]:
CNNmodel = keras.Sequential([
    keras.layers.Conv2D(filters=128, kernel_size=[3,3], strides=strides, use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=128, kernel_size=[3,3], strides=[1,1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=128, kernel_size=[3,3], strides=[1,1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(units=8)
])

In [20]:
Ds_CNNmodel = keras.Sequential([
    keras.layers.Conv2D(filters=256, kernel_size=[3,3], strides=strides, use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.DepthwiseConv2D(kernel_size=[3,3], strides=[1,1],use_bias=False),
    keras.layers.Conv2D(filters=256, kernel_size=[1,1], strides=[1,1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.DepthwiseConv2D(kernel_size=[3,3], strides=[1,1],use_bias=False),
    keras.layers.Conv2D(filters=128, kernel_size=[1,1], strides=[1,1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(units=8)
])

In [21]:
metrics = [tf.metrics.SparseCategoricalAccuracy()]
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [22]:
MLPmodel.compile(optimizer='adam',
              loss=loss,
              metrics=metrics)

CNNmodel.compile(optimizer='adam',
              loss=loss,
              metrics=metrics)

Ds_CNNmodel.compile(optimizer='adam',
              loss=loss,
              metrics=metrics)

In [23]:
cp_callback = keras.callbacks.ModelCheckpoint(
    './callback_test_chkp_MLP/chkp_{epoch:02d}',
    # './callback_test_chkp/chkp_best',
    monitor='val_accuracy',
    verbose=0, 
    save_best_only=False,
    # save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)
MLPmodel.fit(train_ds, epochs=2, callbacks=[cp_callback])

Epoch 1/2
195/200 [============================>.] - ETA: 0s - loss: 1.6491 - sparse_categorical_accuracy: 0.4151WARNING:tensorflow:From e:\env_ML4IOt\lib\site-packages\tensorflow\python\training\tracking\tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./callback_test_chkp_MLP\chkp_01\assets
200/200 [==============================] - 14s 68ms/step - loss: 1.6400 - sparse_categorical_accuracy: 0.4172
Epoch 2/2
200/200 [==============================] - 2s 11ms/step - loss: 1.0943 - sparse_categorical_accuracy: 0.6266


In [24]:
cp_callback = keras.callbacks.ModelCheckpoint(
    './callback_test_chkp_CNN/chkp_{epoch:02d}',
    # './callback_test_chkp/chkp_best',
    monitor='val_accuracy',
    verbose=0, 
    save_best_only=False,
    # save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)
CNNmodel.fit(train_ds, epochs=2, callbacks = cp_callback)

Epoch 1/2
200/200 [==============================] - 21s 106ms/step - loss: 1.5768 - sparse_categorical_accuracy: 0.5407
Epoch 2/2
200/200 [==============================] - 21s 103ms/step - loss: 1.0676 - sparse_categorical_accuracy: 0.6728


In [25]:
train_ds

<ShuffleDataset shapes: ((None, 32, 32, 1), (None,)), types: (tf.float32, tf.int64)>

In [26]:
cp_callback = keras.callbacks.ModelCheckpoint(
    './callback_test_chkp_Ds_CNN/chkp_{epoch:02d}',
    # './callback_test_chkp/chkp_best',
    monitor='val_accuracy',
    verbose=0, 
    save_best_only=False,
    # save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)
Ds_CNNmodel.fit(train_ds, epochs=2, callbacks = cp_callback)

Epoch 1/2
200/200 [==============================] - 26s 132ms/step - loss: 1.6355 - sparse_categorical_accuracy: 0.5570
Epoch 2/2
200/200 [==============================] - 28s 139ms/step - loss: 1.0807 - sparse_categorical_accuracy: 0.6736


In [27]:
test_acc = MLPmodel.evaluate(test_ds, verbose=0)
print('\n accuracy for MLP :', test_acc)

test_acc = CNNmodel.evaluate(test_ds, verbose=0)
print('\n accuracy for CNN :', test_acc)

test_acc = Ds_CNNmodel.evaluate(test_ds, verbose=0)
print('\n accuracy for Ds_CNN :', test_acc)


 accuracy for MLP : [1.0297452211380005, 0.6512500047683716]

 accuracy for CNN : [0.8477685451507568, 0.7225000262260437]

 accuracy for Ds_CNN : [0.9002968072891235, 0.7087500095367432]


In [28]:
run_model = tf.function(lambda x: MLPmodel(x))
concrete_func = run_model.get_concrete_function(tf.TensorSpec([1,32, 32, 1],tf.float32))
MLPmodel.save('MLP', signatures=concrete_func)

INFO:tensorflow:Assets written to: MLP\assets


In [29]:
tf.data.experimental.save(train_ds, './th_train')
tf.data.experimental.save(val_ds, './th_val')
tf.data.experimental.save(test_ds, './th_test')